In [1]:
import glob
import os
# import librosa
# from librosa import display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
%matplotlib inline
plt.style.use('ggplot')
# import pydot
# import graphviz
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'Ubuntu'
plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 11
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 11
plt.rcParams['figure.titlesize'] = 13

/home/admin/anaconda3/envs/py35deeplearning/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
features = np.loadtxt('nn_simple_features.csv', delimiter=',')
labels = np.array(np.loadtxt('nn_simple_labels.csv', delimiter=','), dtype=np.int)

In [15]:
from sklearn.preprocessing import normalize

X_all = features

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(sparse=False)
y_all = enc.fit_transform(labels.reshape(-1, 1))

In [16]:
from sklearn.model_selection import train_test_split
import time

X_not_test, X_test, y_not_rest, y_test = train_test_split(
        X_all, y_all, stratify=y_all, train_size=.85, random_state=round(time.time()))

X_train, X_val, y_train, y_val = train_test_split(
    X_not_test, y_not_rest, stratify=y_not_rest, train_size=.95, random_state=round(time.time()))

/home/admin/anaconda3/envs/py35deeplearning/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [17]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

(11448, 1280)
(11448, 6)
(603, 1280)
(603, 6)
(2127, 1280)
(2127, 6)


In [18]:
## Imports
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.utils import print_summary, plot_model

In [19]:
# Training parameters
batch_size = 32
num_classes = 6
epochs = 40
img_rows, img_cols = 40, 32
print(K.image_data_format())

X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
X_val = X_val.reshape(X_val.shape[0], img_rows, img_cols, 1)

input_shape = (img_rows, img_cols, 1)
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

channels_last
(11448, 40, 32, 1)
(11448, 6)
(603, 40, 32, 1)
(603, 6)
(2127, 40, 32, 1)
(2127, 6)


In [20]:
from keras import regularizers
from keras.layers import BatchNormalization

def build_model():
    # Model definition
    model = Sequential()
    model.add(Conv2D(4, kernel_size=(5, 5),
                     activation='relu',
                     input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(8, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adam(lr=.005),
                  metrics=['accuracy'])
    
    return model

In [21]:
model = build_model()

In [22]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_val, y_val))

Train on 11448 samples, validate on 603 samples
Epoch 1/40
11448/11448 [==============================] - 15s 1ms/step - loss: 1.4670 - acc: 0.4340 - val_loss: 0.8745 - val_acc: 0.6849
Epoch 2/40
11448/11448 [==============================] - 15s 1ms/step - loss: 0.9742 - acc: 0.6377 - val_loss: 0.6555 - val_acc: 0.7678
Epoch 3/40
11448/11448 [==============================] - 13s 1ms/step - loss: 0.8394 - acc: 0.6950 - val_loss: 0.5373 - val_acc: 0.8275
Epoch 4/40
11448/11448 [==============================] - 14s 1ms/step - loss: 0.7432 - acc: 0.7298 - val_loss: 0.4710 - val_acc: 0.8391
Epoch 5/40
11448/11448 [==============================] - 14s 1ms/step - loss: 0.6928 - acc: 0.7492 - val_loss: 0.4275 - val_acc: 0.8375
Epoch 6/40
11448/11448 [==============================] - 15s 1ms/step - loss: 0.6517 - acc: 0.7677 - val_loss: 0.3873 - val_acc: 0.8756
Epoch 7/40
11448/11448 [==============================] - 14s 1ms/step - loss: 0.6116 - acc: 0.7814 - val_loss: 0.3745 - val_acc: 

In [23]:
score = model.evaluate(X_test, y_test, verbose=1)
print()
print('Test loss:', score[0])
print('Test accuracy:', score[1])

2127/2127 [==============================] - 1s 499us/step

Test loss: 0.309219613419097
Test accuracy: 0.8970380818053597
